jupyter notebook --NotebookApp.iopub_data_rate_limit=5368709120

In [40]:
# Reset memory
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [41]:

import warnings
warnings.filterwarnings("ignore")
import joblib
import mlflow

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import plot_confusion_matrix
from scipy.stats import spearmanr
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
import pickle
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [43]:
import xgboost as XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBClassifier

In [48]:
df = pd.read_csv('C:/Users/td/implement_scoring_loan/notebook/data_cleaned.csv')

# Imbalanced Class

In [49]:
# summarize the class distribution
from collections import Counter

target = df.values[:,2]
counter = Counter(target)
for k,v in counter.items():
    per = v / len(target) * 100
    print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))

Class=0, Count=105059, Percentage=34.165%
Class=1, Count=202448, Percentage=65.835%


In [50]:
X=df.drop(['TARGET'],axis=1)
y=df['TARGET']

In [51]:
# Séparation des données en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)

# Équilibrage des données en utilisant SMOTE
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_train_resampled, y_train_resampled= pipeline.fit_resample(X, y)



AttributeError: 'Pipeline' object has no attribute 'fit_resample'

In [ ]:
y_train_resampled.value_counts()/len(y_train_resampled)

# Dummy

Before we can improve our model, we need a baseline.

In [ ]:
clf_dummy = DummyClassifier(random_state=39) 
#establishing random_state for reproducibility
clf_dummy.fit(X_train_resampled, y_train_resampled)
y_pred = clf_dummy.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test,y_pred)

## MLFLOW UI

In [ ]:
def log_mlflow(model):
    # Track params and metrics 
    with mlflow.start_run() as run:
        mlflow.set_tag("model_name", name)
        mlflow.log_param("Colums", paramgrid)
        # Save model to artifacts
        pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', models_[model_]['model'])])
        gscv = GridSearchCV(pipe, param_grid=models_[model_]['params'], cv=cv,scoring='accuracy')
        gscv.fit(X_train_resampled,y_train_resampled)
        mlflow.sklearn.log_model(gscv.best_estimator_, name)
        accuracy = round(accuracy_score(y_test, y_pred), 4)
        auc = roc_auc_score(y_test, y_pred)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("accuracy", accuracy)
        
    mlflow.end_run()

In [26]:
#Score metier
def score_metier(y_test,y_pred):
    cost_fn = 10 ### coût d’un FN est dix fois supérieur au coût d’un FP
    cost_fp = 1
    # Matrice de confusion pour calculer le nombre de FN et de FP ## Voir proba avec seuil
    confusion_matrix = np.array([[np.sum((y_pred == 0) & (y_test == 0)), np.sum((y_pred == 1) & (y_test == 0))],
                                 [np.sum((y_pred == 0) & (y_test == 1)), np.sum((y_pred == 1) & (y_test == 1))]])
    score_metier = cost_fp * confusion_matrix[0,1] + cost_fn * confusion_matrix[1,0]
    return score_metier
    

In [27]:
# Pipeline for numerical and categorical pipelines

preprocessor=PowerTransformer(method='yeo-johnson',standardize=True)


In [28]:
param_log_reg = {'model__penalty': ['l1', 'l2'],
    'model__C': np.logspace(-4, 4, 20),
    'model__solver': ['liblinear', 'saga']
}

param_Xgboost = {
    'model__n_estimators': [50,100],
    'model__max_depth': [3, 4],
    'model__learning_rate': [0.01, 0.2]
}

In [29]:
# Dictionnaire de classifieurs 
models_ = { 'Logistic Regression':{'model':LogisticRegression(),'params':param_log_reg},
            'Xgboost':{'model': XGBClassifier(), 'params':param_Xgboost}
              }

In [30]:
cv = StratifiedKFold(n_splits=5)

In [ ]:
# Dictionnaire de classifieurs 
models_ = { 'Logistic Regression':{'model':LogisticRegression(),'params':param_log_reg},
            'Xgboost':{'model': XGBClassifier(), 'params':param_Xgboost}
              }
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.sklearn.autolog()

for model_ in models_.keys():
    experiment_name = model_
    mlflow.set_experiment(experiment_name)
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', models_[model_]['model'])])
    gscv = GridSearchCV(pipe, param_grid=models_[model_]['params'], cv=cv,scoring='accuracy')
    gscv.fit(X_train_resampled,y_train_resampled)
    
    with mlflow.start_run(run_name=model_):
        mlflow.sklearn.log_model(gscv.best_estimator_, model_)
        
        # Evaluate the model on the test data

        probabilities = pipe.predict_proba(X_test)
        #Score metier
        threshold = 0.7
        y_pred = np.where(probabilities[:, 1] >= threshold, 1, 0)

        cost_fn = 10  # coût d’un FN est dix fois supérieur au coût d’un FP
        cost_fp = 1
        # Matrice de confusion pour calculer le nombre de FN et de FP ## Voir proba avec seuil
        confusion_matrix = np.array([[np.sum((y_pred == 0) & (y_test == 0)), np.sum((y_pred == 1) & (y_test == 0))],
                                     [np.sum((y_pred == 0) & (y_test == 1)), np.sum((y_pred == 1) & (y_test == 1))]])
        score_metier = cost_fp * confusion_matrix[0,1] + cost_fn * confusion_matrix[1,0]

        # Log the model
        mlflow.sklearn.log_model(pipe, "model")

        # score_metier as metrics
        accuracy = round(accuracy_score(y_test, y_pred), 4)
        auc = roc_auc_score(y_test, y_pred)
        mlflow.log_metric("auc", auc)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("score_metier", score_metier)

mlflow.end_run()

2023/02/20 18:10:57 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '55af88ee03ee43f0b67d55e97b4838d6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


In [ ]:
# ## Erase experiment
# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# experiments = client.search_experiments("DummyClassifier")
# for experiment in experiments:
#     client.delete_experiment(experiment.experiment_id)


## STREAMLIT AND FASTAPI

In [ ]:
# Saving the selected model

model_saved=LogisticRegression()

In [ ]:
# Saving model for fastapi

joblib.dump(model_saved, './scoring_loan.joblib')

### Unit Test


In [ ]:
# from my_module import score_metier
import unittest
class TestScoreMetier(unittest.TestCase):

    def test_score_metier(self):
        y_test = np.array([0, 1, 0, 1, 0, 1])
        y_pred = np.array([0, 1, 1, 1, 0, 0])
        expected_score = 20 # Calculé manuellement pour les entrées ci-dessus
        self.assertEqual(score_metier(y_test, y_pred), expected_score)

In [29]:
df.to_csv('C:/Users/td/implement_scoring_loan/notebook/data.csv', index=False)

In [ ]:
model_saved.